In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
data = pd.read_csv('ecommerce_data.csv')

# Display the first few rows
data.head()

# Get dataset summary
data.info()

# Check for missing values
data.isnull().sum()

# Fill or drop missing values
data.fillna('', inplace=True)

# Clean text data
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

# Assuming 'product' column is the text column you want to clean
data['cleaned_text'] = data['product'].apply(clean_text)

# Convert categorical data to numerical
categorical_columns = ['user id', 'product id']
data = pd.get_dummies(data, columns=categorical_columns)

# Convert 'rating' to binary target
data['binary_rating'] = (data['rating'] >= 1000).astype(int)

# Check class distribution
print(data['binary_rating'].value_counts())

X = data.drop(['rating', 'product', 'cleaned_text', 'binary_rating'], axis=1)
y = data['binary_rating']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the trained model
model = load_model('model_weights.h5')

# Make predictions
predictions = model.predict(X_test)

# Adjust the threshold
threshold = 0.5  # Start with default, then adjust if needed
predictions = (predictions > threshold).astype(int)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig('confusion_matrix.png')
plt.show()
